In [1]:
from pyspark.sql import SparkSession

spark: SparkSession = SparkSession.builder \
                    .master('local') \
                    .appName('first_spark_application') \
                    .getOrCreate()


24/11/13 14:02:46 WARN Utils: Your hostname, pmurugesan-lnx1 resolves to a loopback address: 127.0.1.1; using 192.168.1.33 instead (on interface wlo1)
24/11/13 14:02:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/13 14:02:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/13 14:02:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/13 14:02:47 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
data = spark.read.csv('test1.csv',header=True,inferSchema=True)
data.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [3]:
# Group the indepentent features.
# Here we are not keeping the separate train_x and train_y dataframe
# Instead we are going to group all the independent feature as list[feature] --> [Age,Name] likeof
# Now this will be my new coulmn

from pyspark.ml.feature import VectorAssembler

feat_ensembler = VectorAssembler(inputCols=["age", "Experience"], outputCol="Independent Features")

In [4]:
output = feat_ensembler.transform(data)
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [5]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [7]:
fin_data = output.select(["Independent Features", "Salary"])
fin_data.show()


+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [11]:
# Regression
from pyspark.ml.regression import LinearRegression

train_data, test_data = fin_data.randomSplit([0.75, 0.25])

reg = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
fitted = reg.fit(train_data)

24/11/13 14:11:13 WARN Instrumentation: [1e507a74] regParam is zero, which might cause numerical instability and overfitting.


In [12]:
fitted.coefficients

DenseVector([-102.53, 1688.6818])

In [13]:
fitted.intercept

16470.03994673731

In [15]:
resul = fitted.evaluate(test_data)

In [16]:
resul.predictions.show()


+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [30.0,8.0]| 25000|26903.59520639148|
+--------------------+------+-----------------+



In [17]:
resul.meanAbsoluteError,resul.meanSquaredError


(1903.595206391481, 3623674.709796625)